<a href="https://colab.research.google.com/github/faizarindra/SistemPendukungKeputusan/blob/main/Tubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Pendukung Keputusan Pemilihan Perbaikan Jalan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from collections import defaultdict

**Kriteria**


1.   Lalu lintas harian rata-rata (LHR)
2.   Laporan Masyarakat (LM)



In [ ]:
 kriteria = pd.DataFrame({'Bobot':[0.15 , 0.05 , 0.2 , 0.13 , 0.08 , 0.12 , 0.05 , 0.1 , 0.05 , 0.07 ],
                          'Cost/Benefit': ["benefit", "benefit", "benefit", "cost","benefit","cost","benefit","cost","benefit","cost"]},
             index = ["Luas Kerusakan","Umur Jalan","LHR","Kondisi Jalan","Bangunan Pelengkap","Biaya Pekerjaan","LM","Tipe Jalan","Luas Tambalan","Kepadatan Penduduk"])
 kriteria

,Bobot,Cost/Benefit
Luas Kerusakan,0.15,benefit
Umur Jalan,0.05,benefit
LHR,0.20,benefit
Kondisi Jalan,0.13,cost
Bangunan Pelengkap,0.08,benefit
Biaya Pekerjaan,0.12,cost
LM,0.05,benefit
Tipe Jalan,0.10,cost
Luas Tambalan,0.05,benefit
Kepadatan Penduduk,0.07,cost


In [ ]:
 alternatif = pd.DataFrame({'Luas Kerusakan':[45300,57000,56700,45985,67730,69100,57950,59244,43694,50989,43523,59384,42547,67836,32342,65333,54322,54233,42343,68906],
                          'Umur Jalan': [6,7,6,5,9,9,7,6,8,5,5,9,7,5,6,6,7,8,5,8],
                          'LHR': [460,800,500,581,783,552,573,582,500,411,517,696,625,875,373,272,432,542,732,673],
                          'Kondisi Jalan':[2,3,3,2,2,1,3,1,1,2,3,2,1,3,2,3,2,3,3,1],
                          'Kondisi Bangunan Pelengkap':[4,4,4,4,3,5,2,4,2,3,2,5,5,4,3,4,3,1,2,5],
                          'Biaya Pekerjaan':[56730000,67900000,47875300,86966000,77900000,69168000,86600000,73595000,48910000,54024595,43245700,45325636,70600000,65004000,50600000,60700000,41000000,30600710,70300600,77000000],
                          'LM': [2,2,3,1,2,2,1,1,1,3,2,1,2,2,3,1,1,2,1,2],
                          'Tipe Jalan':[3,1,2,2,1,3,2,1,3,2,3,1,3,2,1,3,2,1,3,3],
                          'Luas Tambalan':[210,290,277,153,433,225,236,198,166,235,149,304,269,287,257,110,322,221,215,343],
                          'KP':[2,3,2,2,1,2,1,2,3,1,3,1,2,2,1,2,3,2,2,1]},
             index = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20"])
alternatif

,Luas Kerusakan,Umur Jalan,LHR,Kondisi Jalan,Kondisi Bangunan Pelengkap,Biaya Pekerjaan,LM,Tipe Jalan,Luas Tambalan,KP
A1,45300,6,460,2,4,56730000,2,3,210,2
A2,57000,7,800,3,4,67900000,2,1,290,3
A3,56700,6,500,3,4,47875300,3,2,277,2
A4,45985,5,581,2,4,86966000,1,2,153,2
A5,67730,9,783,2,3,77900000,2,1,433,1
A6,69100,9,552,1,5,69168000,2,3,225,2
A7,57950,7,573,3,2,86600000,1,2,236,1
A8,59244,6,582,1,4,73595000,1,1,198,2
A9,43694,8,500,1,2,48910000,1,3,166,3
A10,50989,5,411,2,3,54024595,3,2,235,1


In [ ]:


def read_csv(filename):
    df = pd.read_csv(filename)
    return df

def saw(df, weights, criteria, criterion_types):
    # Normalisasi matriks keputusan
    normalized_df = df.copy()
    for criterion, criterion_type in zip(criteria, criterion_types):
        column = df[criterion]
        if criterion_type == 'cost':
            normalized_df[criterion] = (column.min())  / column
        elif criterion_type == 'benefit':
            normalized_df[criterion] = column / (column.max() )
    print("Matriks Normalisasi:")
    print(normalized_df)
    # Mengalikan matriks keputusan dengan bobot
    weighted_df = normalized_df.copy()
    for criterion, weight in zip(criteria, weights):
        weighted_df[criterion] *= weight

    # Menjumlahkan nilai dari setiap alternatif
    weighted_df['Total'] = weighted_df.sum(axis=1)
    print("Pembobotan: \n",weighted_df)
    # Mengurutkan alternatif berdasarkan nilai total secara menurun
    sorted_df = weighted_df.sort_values(by='Total', ascending=False)
    weighted_df['Ranking'] = sorted_df['Total'].rank(ascending=False).astype(int)
    print("Perankingan :\n",weighted_df)
    return sorted_df

# Fungsi untuk memasukkan data CSV
def input_csv_data():
    criteria = []
    data = {}

    # Memasukkan jumlah kriteria
    num_criteria = int(input("Masukkan jumlah kriteria: "))

    # Memasukkan nama kriteria dan jenis (cost/benefit)
    for i in range(num_criteria):
        criterion = input("Masukkan nama kriteria ke-%d: " % (i+1))
        criteria.append(criterion)
        criterion_type = input("Jenis kriteria (cost/benefit) untuk kriteria %s: " % criterion)
        data[criterion] = []
        data['Type ' + criterion] = criterion_type

    # Memasukkan data alternatif
    num_alternatives = int(input("Masukkan jumlah alternatif: "))
    for j in range(num_alternatives):
        print("Masukkan data untuk Alternatif %d:" % (j+1))
        for criterion in criteria:
            value = float(input("Masukkan nilai untuk kriteria %s: " % criterion))
            data[criterion].append(value)

    # Membuat DataFrame dari data yang dimasukkan
    df = pd.DataFrame(data)

    return df, criteria

# Memasukkan data CSV
dataset, criteria = input_csv_data()

# Definisikan bobot untuk masing-masing kriteria
weights = [0.15, 0.05, 0.2,0.13,0.08,0.12,0.05,0.1,0.05,0.07]

# Definisikan jenis kriteria (cost/benefit)
criterion_types = [dataset['Type ' + criterion].iloc[0] for criterion in criteria]

# Menggunakan metode SAW untuk menghitung skor alternatif
result = saw(dataset, weights, criteria, criterion_types)

# Menampilkan hasil
print(result)

KeyboardInterrupt: ignored

Dataset

In [ ]:
alternatif = ["Jalan Suhat", "Jalan Sudimoro", "Jalan Sawojajar", "Jalan Mt Haryono","Jalan Songgoriti","Jalan Merjosari","Jalan Ikan hiu","Jalan China","Jalan Dolly","Jalan Pasar Kembang","Jalan Sragen","Jalan tuban","Jalan Solo","Jalan Kendal","Jalan Rio","Jalan Danau Toba","Jalan Matos","Jalan Amerika","Jalan Meksiko","Jalan Lowokwaru"]
kriteria = ["Luas Kerusakan","Umur Jalan","LHR","Kondisi Jalan","Bangunan Pelengkap","Biaya Pekerjaan","LM","Tipe Jalan","Luas tambalan","Kepadatan Penduduk"]
costbenefit = ["benefit", "benefit", "benefit", "cost","benefit","cost","benefit","cost","benefit","cost"]
kepentingan = [0.15 , 0.05 , 0.2 , 0.13 , 0.08 , 0.12 , 0.05 , 0.1 , 0.05 , 0.07 ]
alternatifkriteria = [[45300,	6,	460,	2,	4,	56730000,	2,	3	,210	,2],
 [57000,	7,	800,	3,	4,	67900000,	2,	1	,290	,3],
  [56700,	6,	500,	3,	4,	47875300,	3,	2	,277	,2],
   [45985,	5,	581,	2,	4,	86966000,	1,	2	,153	,2],
    [67730,	9,	783,	2,	3,	77900000,	2,	1	,433	,1],
     [69100,	9,	552,	1,	5,	69168000,	2,	3	,225	,2],
      [57950,	7,	573,	3,	2,	86600000,	1,	2	,236	,1],
                      [59244,	6,	582,	1,	4,	73595000,	1,	1	,198	,2],
                      [43694,	8,	500,	1,	2,	48910000,	1,	3	,166	,3],
                      [50989,	5,	411,	2,	3,	54024595,	3,	2	,235	,1],
                      [43523,	5,	517,	3,	2,	43245700,	2,	3	,149	,3],
                      [59384,	9,	696,	2,	5,	45325636,	1,	1	,304	,1],
                      [42547,	7,	625,	1,	5,	70600000,	2,	3	,269	,2],
                      [67836,	5,	875,	3,	4,	65004000,	2,	2	,287	,2],
                      [32342,	6,	373,	2,	3,	50600000,	3,	1	,257	,1],
                      [65333,	6,	272,	3,	4,	60700000,	1,	3	,110	,2],
                      [54322,	7,	432,	2,	3,	41000000,	1,	2	,322	,3],
                      [54233,	8,	542,	3,	1,	30600710,	2,	1	,221	,2],
                      [42343,	5,	732,	3,	2,	70300600,	1,	3	,215	,2],
                      [68906,	8,	673,	1,	5,	77000000,	2,	3	,343	,1]]

**Normalisasi Cost/Benefit**

In [ ]:
pembagi = []
for i in range(len(kriteria)):
     pembagi.append(0)
     for j in range(len(alternatif)):
             if costbenefit[i] == 'cost':
                     if j == 0:
                             pembagi[i] = alternatifkriteria[j][i]
                     else:
                             if pembagi[i] > alternatifkriteria[j][i]:
                                     pembagi[i] = alternatifkriteria[j][i]
             else:   #if costbenefit[i] == 'benefit:
                     if j == 0:
                             pembagi[i] = alternatifkriteria[j][i]
                     else:
                             if pembagi[i] < alternatifkriteria[j][i]:
                                     pembagi[i] = alternatifkriteria[j][i]


normalisasi = []
for i in range(len(alternatif)):
     normalisasi.append([])
     for j in range(len(kriteria)):
             normalisasi[i].append(0)
             if costbenefit[j] == 'cost':
                     normalisasi[i][j] = pembagi[j] / alternatifkriteria[i][j]
             else:           #if costbenefit[j] == 'benefit':
                     normalisasi[i][j] = alternatifkriteria[i][j] / pembagi[j]


df = pd.DataFrame(normalisasi,columns=["C1","C2","C3","C4","C5","C6","C7","C8","C9","C10"],index = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20"])
df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
A1,0.655572,0.666667,0.525714,0.500000,0.8,0.539410,0.666667,0.333333,0.484988,0.500000
A2,0.824891,0.777778,0.914286,0.333333,0.8,0.450673,0.666667,1.000000,0.669746,0.333333
A3,0.820550,0.666667,0.571429,0.333333,0.8,0.639175,1.000000,0.500000,0.639723,0.500000
A4,0.665485,0.555556,0.664000,0.500000,0.8,0.351870,0.333333,0.500000,0.353349,0.500000
A5,0.980174,1.000000,0.894857,0.500000,0.6,0.392820,0.666667,1.000000,1.000000,1.000000
A6,1.000000,1.000000,0.630857,1.000000,1.0,0.442411,0.666667,0.333333,0.519630,0.500000
A7,0.838640,0.777778,0.654857,0.333333,0.4,0.353357,0.333333,0.500000,0.545035,1.000000
A8,0.857366,0.666667,0.665143,1.000000,0.8,0.415799,0.333333,1.000000,0.457275,0.500000
A9,0.632330,0.888889,0.571429,1.000000,0.4,0.625653,0.333333,0.333333,0.383372,0.333333
A10,0.737902,0.555556,0.469714,0.500000,0.6,0.566422,1.000000,0.500000,0.542725,1.000000


**Pembobotan**

In [ ]:
hasil = []
for i in range(len(alternatif)):
     hasil.append(0)
     for j in range(len(kriteria)):
              normalisasi[i][j]*=kepentingan[j]
              hasil[i] += normalisasi[i][j]

            #  hasil[i] = hasil[i] + (normalisasi[i][j] * kepentingan[j])
# hasil1 = pd.DataFrame({"Nilai Total":hasil},index=["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20"])
# hasil1
df = pd.DataFrame(normalisasi,columns=["C1","C2","C3","C4","C5","C6","C7","C8","C9","C10"],index = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20"])
df


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
A1,0.098336,0.033333,0.105143,0.065000,0.064,0.064729,0.033333,0.033333,0.024249,0.035000
A2,0.123734,0.038889,0.182857,0.043333,0.064,0.054081,0.033333,0.100000,0.033487,0.023333
A3,0.123082,0.033333,0.114286,0.043333,0.064,0.076701,0.050000,0.050000,0.031986,0.035000
A4,0.099823,0.027778,0.132800,0.065000,0.064,0.042224,0.016667,0.050000,0.017667,0.035000
A5,0.147026,0.050000,0.178971,0.065000,0.048,0.047138,0.033333,0.100000,0.050000,0.070000
A6,0.150000,0.050000,0.126171,0.130000,0.080,0.053089,0.033333,0.033333,0.025982,0.035000
A7,0.125796,0.038889,0.130971,0.043333,0.032,0.042403,0.016667,0.050000,0.027252,0.070000
A8,0.128605,0.033333,0.133029,0.130000,0.064,0.049896,0.016667,0.100000,0.022864,0.035000
A9,0.094849,0.044444,0.114286,0.130000,0.032,0.075078,0.016667,0.033333,0.019169,0.023333
A10,0.110685,0.027778,0.093943,0.065000,0.048,0.067971,0.050000,0.050000,0.027136,0.070000


In [ ]:
hasil1 = pd.DataFrame({"Nilai Total":hasil},index=["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20"])
hasil1

,Nilai Total
A1,0.556457
A2,0.697048
A3,0.621722
A4,0.550959
A5,0.789469
A6,0.716909
A7,0.577311
A8,0.713393
A9,0.583160
A10,0.610513


**Perangkingan**

In [ ]:
alternatifrangking = []
hasilrangking = []
for i in range(len(alternatif)):
     hasilrangking.append(hasil[i])
     alternatifrangking.append(alternatif[i])

for i in range(len(alternatif)):
     for j in range(len(alternatif)):
             if j > i:
                     if hasilrangking[j] > hasilrangking[i]:
                             tmphasil = hasilrangking[i]
                             tmpalternatif = alternatifrangking[i]
                             hasilrangking[i] = hasilrangking[j]
                             alternatifrangking[i]=alternatifrangking[j]
                             hasilrangking[j] = tmphasil
                             alternatifrangking[j] = tmpalternatif



af = pd.DataFrame({"Ranking":alternatifrangking,"Nilai":hasilrangking}, index=["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"])

af

,Ranking,Nilai
1,Jalan Songgoriti,0.789469
2,Jalan tuban,0.785781
3,Jalan Lowokwaru,0.781815
4,Jalan Merjosari,0.716909
5,Jalan China,0.713393
6,Jalan Sudimoro,0.697048
7,Jalan Kendal,0.690332
8,Jalan Solo,0.668847
9,Jalan Amerika,0.659244
10,Jalan Rio,0.624045


In [ ]:
import pandas as pd

def read_csv(filename):
    df = pd.read_csv(filename)
    return df

def saw(df, weights, criteria, criterion_types):
    # Normalisasi matriks keputusan
    normalized_df = df.copy()
    for criterion, criterion_type in zip(criteria, criterion_types):
        column = df[criterion]
        if criterion_type == 'cost':
            normalized_df[criterion] = (column.min())  / column
        elif criterion_type == 'benefit':
            normalized_df[criterion] = column / (column.max() )
    print("Matriks Normalisasi:")
    print(normalized_df)
    # Mengalikan matriks keputusan dengan bobot
    weighted_df = normalized_df.copy()
    for criterion, weight in zip(criteria, weights):
        weighted_df[criterion] *= weight
    print("Matriks Terbobot:\n", weighted_df)
    # Menjumlahkan nilai dari setiap alternatif
    weighted_df['Total'] = weighted_df.sum(axis=1)
    print("Total: \n",weighted_df)
    # Mengurutkan alternatif berdasarkan nilai total secara menurun
    sorted_df = weighted_df.sort_values(by='Total', ascending=False)
    weighted_df['Ranking'] = sorted_df['Total'].rank(ascending=False).astype(int)

    print("Perankingan :\n",weighted_df)
    return sorted_df

# Baca dataset dari file CSV

dataset = read_csv('data.csv')

# Definisikan bobot untuk masing-masing kriteria
weights = [0.15, 0.05, 0.2,0.13,0.08,0.12,0.05,0.1,0.05,0.07]

# Definisikan kriteria yang digunakan
# criteria = ['Luas Kerusakan', 'Umur Jalan', 'LHR','Kondisi Jalan','Bangunan Pelengkap','Biaya Pekerjaan','LM','Tipe Jalan','Luas Tambalan','Kepadatan Penduduk']
criteria = ['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10']
# Definisikan jenis kriteria (cost/benefit)
criterion_types = ['benefit', 'benefit', 'benefit','cost','benefit','cost','benefit','cost','benefit','cost']

# Menggunakan metode SAW untuk menghitung skor alternatif
result = saw(dataset, weights, criteria, criterion_types)

# Menampilkan hasil
# print("Perankingan :\n",result)

Matriks Normalisasi:
   Alternatif        C1        C2        C3        C4   C5        C6  \
0          A1  0.659338  0.777778  0.520548  0.500000  0.8  0.539168   
1          A2  0.820671  0.888889  0.896119  0.500000  1.0  0.453050   
2          A3  0.822456  0.666667  0.547945  0.333333  0.8  0.639309   
3          A4  0.667378  0.777778  0.665525  0.333333  1.0  0.351914   
4          A5  0.985821  0.777778  0.890411  0.333333  0.8  0.387842   
5          A6  0.999898  0.888889  0.627854  0.500000  0.8  0.442418   
6          A7  0.840698  1.000000  0.650685  0.500000  0.8  0.354175   
7          A8  0.859782  0.888889  0.662100  1.000000  0.6  0.415810   
8          A9  0.634116  0.777778  0.568493  0.333333  0.4  0.625781   
9         A10  0.739954  0.666667  0.468037  0.500000  0.2  0.566422   
10        A11  0.631634  0.555556  0.591324  0.333333  0.2  0.707599   
11        A12  0.861770  0.777778  0.793379  0.333333  0.6  0.675130   
12        A13  0.617426  0.888889  0.711187

<ipython-input-10-0489aeb4437c>:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  weighted_df['Total'] = weighted_df.sum(axis=1)


In [ ]:
import pandas as pd

def read_csv(filename):
    df = pd.read_csv(filename)
    return df

def saw(df, weights, criteria, criterion_types):
    # Normalisasi matriks keputusan
    normalized_df = df.copy()
    for criterion, criterion_type in zip(criteria, criterion_types):
        column = df[criterion]
        if criterion_type == 'cost':
            normalized_df[criterion] = (column.min())  / column
        elif criterion_type == 'benefit':
            normalized_df[criterion] = column / (column.max() )
    print("Matriks Normalisasi:")
    print(normalized_df)
    # Mengalikan matriks keputusan dengan bobot
    weighted_df = normalized_df.copy()
    for criterion, weight in zip(criteria, weights):
        weighted_df[criterion] *= weight
    print("Matriks Terbobot:\n", weighted_df)
    # Menjumlahkan nilai dari setiap alternatif
    weighted_df['Total'] = weighted_df.sum(axis=1)
    print("Total: \n",weighted_df)
    # Mengurutkan alternatif berdasarkan nilai total secara menurun
    sorted_df = weighted_df.sort_values(by='Total', ascending=False)
    weighted_df['Ranking'] = sorted_df['Total'].rank(ascending=False).astype(int)

    print("Perankingan :\n",weighted_df)
    return sorted_df

# Baca dataset dari file CSV

dataset = read_csv('data.csv')

# Definisikan bobot untuk masing-masing kriteria
weights = [0.15, 0.05, 0.2,0.13,0.08,0.12,0.05,0.1,0.05,0.07]

# Definisikan kriteria yang digunakan
# criteria = ['Luas Kerusakan', 'Umur Jalan', 'LHR','Kondisi Jalan','Bangunan Pelengkap','Biaya Pekerjaan','LM','Tipe Jalan','Luas Tambalan','Kepadatan Penduduk']
criteria = ['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10']
# Definisikan jenis kriteria (cost/benefit)
criterion_types = ['benefit', 'benefit', 'benefit','cost','benefit','cost','benefit','cost','benefit','cost']

# Menggunakan metode SAW untuk menghitung skor alternatif
result = saw(dataset, weights, criteria, criterion_types)

# Menampilkan hasil
# print("Perankingan :\n",result)

In [ ]:
def borda_count(ranking):
    num_choices = len(ranking[0])
    borda_scores = [0] * num_choices


    # Menghitung skor Borda
    for rank in ranking:
        for i in range(num_choices):
            borda_scores[rank[i] - 1] += (num_choices - i - 1)

    return borda_scores

# Contoh input peringkat
def ranking_list(values):
    sorted_values = sorted(values, reverse=True)
    rankings = [sorted_values.index(x) + 1 for x in values]
    return rankings
ranking = [
    [5,12,20,6,8,2,14,13,18,15,3,10,17,9,7,1,4,19,11,16],  # Pilihan 1 di peringkat 2, pilihan 2 di peringkat 3, ...
    [14,2,13,5,20,19,7,18,3,8,12,6,15,9,17,1,11,4,10,16]
]

borda_scores = borda_count(ranking)
bd = ranking_list(borda_scores)
# Menampilkan skor Borda
for choice, score in enumerate(borda_scores):
    print(f'Alternatif {choice+1}: {score} poin')


# for choice, i in enumerate(bd):
#     print(f'Alternatif {choice+1} mendapatkan peringkat {bd[i]}')






Alternatif 1: 8 poin
Alternatif 2: 32 poin
Alternatif 3: 20 poin
Alternatif 4: 5 poin
Alternatif 5: 35 poin
Alternatif 6: 24 poin
Alternatif 7: 18 poin
Alternatif 8: 25 poin
Alternatif 9: 12 poin
Alternatif 10: 9 poin
Alternatif 11: 4 poin
Alternatif 12: 27 poin
Alternatif 13: 29 poin
Alternatif 14: 32 poin
Alternatif 15: 17 poin
Alternatif 16: 0 poin
Alternatif 17: 12 poin
Alternatif 18: 23 poin
Alternatif 19: 16 poin
Alternatif 20: 32 poin


In [ ]:
borda = pd.DataFrame({'Ranking': [14,2,8,15,1,6,9,5,12,13,16,4,3,2,10,17,12,7,11,2]},
             index = ["Jalan Suhat", "Jalan Sudimoro", "Jalan Sawojajar", "Jalan Mt Haryono","Jalan Songgoriti","Jalan Merjosari","Jalan Ikan hiu","Jalan China","Jalan Dolly","Jalan Pasar Kembang","Jalan Sragen","Jalan tuban","Jalan Solo","Jalan Kendal","Jalan Rio","Jalan Danau Toba","Jalan Matos","Jalan Amerika","Jalan Meksiko","Jalan Lowokwaru"])
borda

,Ranking
Jalan Suhat,14
Jalan Sudimoro,2
Jalan Sawojajar,8
Jalan Mt Haryono,15
Jalan Songgoriti,1
Jalan Merjosari,6
Jalan Ikan hiu,9
Jalan China,5
Jalan Dolly,12
Jalan Pasar Kembang,13
